In [ ]:
from ipydatagrid import DataGrid, Filter, Sort, TextRenderer, BarRenderer, Predicate

In [ ]:
from json import load

with open('./cars.json') as fobj:  
    data = load(fobj)

In [ ]:
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale

horsepower_operator = Predicate('value', '<', 150, 'red', 'green')

renderers = {
    'Acceleration': BarRenderer(
        horizontal_alignment='center',
        bar_color=ColorScale(min=0, max=20, scheme='viridis'),
        value=LinearScale(min=0, max=20)
    ),
    'Displacement': TextRenderer(
        text_color=ColorScale(min=97, max=455), 
        font=Predicate('value', '>', 400, '16px sans-serif', '12px sans-serif')
    ),
    'Horsepower': TextRenderer(
        background_color=horsepower_operator
    ),
    'Name': TextRenderer(
        background_color=[Predicate('value', 'contains', 'chevrolet', 'green'), Predicate('value', 'contains', 'torino', 'yellow')]
    ),
    'Origin': TextRenderer(
        background_color=OrdinalColorScale(domain=['USA', 'Japan', 'Europe']),
        horizontal_alignment=Predicate('value', '=', 'USA', 'right')
    ),
    'Year': TextRenderer(background_color='green'),
}

datagrid = DataGrid(data=data, base_row_size=32, base_column_size=100, renderers=renderers)
datagrid

In [ ]:
horsepower_operator.output_if_true = 'orange'

In [ ]:
datagrid.transform([Filter('Origin', '=', 'Europe'), Sort('Displacement')])

In [ ]:
datagrid.revert()

In [ ]:
datagrid.transform([Filter('Origin', '=', 'USA'), Filter('Acceleration', '<', 13), Sort('Acceleration')])

# Perfomance test: A million cells with ipydatagrid

In [ ]:
from random import uniform

def create_random_data(n_rows=100, n_columns=100):
    data = {'data': [], 'schema': {}}

    data['data'] = [[uniform(0, 1) for c in range(n_columns)] for r in range(n_rows)]
    data['schema']['fields'] = [{'name': str(c), type:'number'} for c in range(n_columns)]
    
    return data

def update_random_data(old_data):
    data = {'data': [], 'schema': {}}
    
    n_columns = len(old_data['data'])
    n_rows = len(old_data['data'][0])

    data['data'] = [[uniform(-0.1, 0.1) + old_data['data'][r][c]  for c in range(n_columns)] for r in range(n_rows)]
    data['schema']['fields'] = [{'name': str(c), type:'number'} for c in range(n_columns)]
    
    return data

In [ ]:
huge_data = create_random_data(1000, 1000)

conditional_renderer = Predicate('value', '<', 0.5, '#fc8403', 'white')

conditional_huge_datagrid = DataGrid(data=huge_data, default_renderer=TextRenderer(background_color=conditional_renderer))
conditional_huge_datagrid

In [ ]:
from ipywidgets import FloatSlider, Dropdown, ColorPicker, HBox, link

operator_dropdown = Dropdown(
    options=['<', '>', '='],
    value=conditional_renderer.operator
)
link((conditional_renderer, 'operator'), (operator_dropdown, 'value'))

reference_slider = FloatSlider(value=conditional_renderer.reference_value, min=0, max=1)
link((conditional_renderer, 'reference_value'), (reference_slider, 'value'))

output_colorpicker = ColorPicker(
    value=conditional_renderer.output_if_true
)
link((conditional_renderer, 'output_if_true'), (output_colorpicker, 'value'))

HBox((operator_dropdown, reference_slider, output_colorpicker))

In [ ]:
conditional_renderer.output_if_false = 'red'

In [ ]:
conditional_renderer.output_if_false = None

## Bar renderer

In [ ]:
from bqplot import LinearScale, ColorScale
from ipydatagrid import DataGrid, BarRenderer

linear_scale = LinearScale(min=0, max=1)
color_scale = ColorScale(min=0, max=1)
bar_renderer = BarRenderer(bar_color=color_scale, value=linear_scale, bar_horizontal_alignment='center')

huge_datagrid = DataGrid(data=create_random_data(), default_renderer=bar_renderer)

In [ ]:
huge_datagrid

In [ ]:
from ipywidgets import FloatSlider, link

slider = FloatSlider(description='Scale: ', value=linear_scale.max, min=0, max=1)
link((color_scale, 'min'), (slider, 'value'))
link((linear_scale, 'min'), (slider, 'value'))

slider

In [ ]:
color_scale.scheme = 'magma'

In [ ]:
huge_datagrid.data = update_random_data(huge_datagrid.data)